In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CoolPropWarning
from hvac.heat_exchanger.recuperator.fintube.continuous_fin import (
    PlainFinTubeAirToWaterCounterFlowHeatExchanger
)


warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity

# Air-to-Water Cooling Coil (Part 1)

Given an air-to-water cooling coil and the "input conditions" to this coil, we want to determine the performance of this cooling coil for this specific set of input conditions.

**Given:**<br>
- an air-to-water cooling coil with known dimensions and heat exchanger core geometry
- the state of the air entering the coil
- the volume flow rate of air through the coil
- the state of the water entering the coil
- the volume flow rate of water through the coil

**Find:**<br>
- the heat transfer rate from the air to the water stream
- the state of the air leaving the coil
- the state of the water leaving the coil
- the air-side pressure loss across the coil
- the water-side pressure loss across the coil

**Define the dimensions and the heat exchanger core geometry of the cooling coil:**

In [2]:
AirCoil = PlainFinTubeAirToWaterCounterFlowHeatExchanger

air_coil = AirCoil(
    width=Q_(900, 'mm'),
    height=Q_(180, 'mm'),
    num_rows=3,
    pitch_trv=Q_(25.4, 'mm'),
    pitch_lon=Q_(22.0, 'mm'),
    d_i=Q_(8.422, 'mm'),
    d_o=Q_(10.200, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    fin_density=1 / Q_(3.175, 'mm'),
    num_circuits=None
)

**Specify the known input conditions on the air side:**
- dry bulb temperature entering air: 27 °C
- wet bulb temperature entering air: 19 °C
- volume flow rate of air measured at the entry: 850 m³/h

In [3]:
air_in = HumidAir(Tdb=Q_(27, 'degC'), Twb=Q_(19, 'degC'))
air_V_dot = Q_(850.0, 'm ** 3 / hr')

air_m_dot = air_in.rho * air_V_dot

**Specify the known input conditions on the waterside:**
- average water system pressure: 2 bar
- temperature of water entering the coil: 7 °C
- volume flow rate of water measured at the entry: 853.02 L/h

In [4]:
Water = Fluid('Water')

water_in = Water(T=Q_(7, 'degC'), P=Q_(2, 'bar'))
water_V_dot = Q_(853.02, 'L / hr')

water_m_dot = water_in.rho * water_V_dot

**Set the known operating conditions on the air coil:**

In [5]:
air_coil.set_operating_conditions(
    air_in=air_in,
    water_in=water_in,
    air_m_dot=air_m_dot,
    water_m_dot=water_m_dot
)

**Get the answers:**

In [6]:
result = air_coil.rate()

In [7]:
print(
    "air-side surface condition = "
    f"{result['air_surface_condition']}",
    "air out = "
    f"{result['air_out'].Tdb.to('degC'):~P.2f} DB, "
    f"{result['air_out'].RH.to('pct'):~P.0f} RH",
    "water out = "
    f"{result['water_out'].T.to('degC'):~P.2f}, "
    f"{result['water_out'].P.to('bar'):~P.3f}",
    "heat transfer rate = "
    f"{result['Q_dot'].to('kW'):~P.3f}",
    "heat exchanger effectiveness = "
    f"{result['eps']}",
    "air-side pressure drop = "
    f"{result['dP_air'].to('Pa'):~P.3f}",
    "water-side pressure drop = "
    f"{result['dP_water'].to('Pa'):~P.3f}",
    sep='\n', end='\n\n'
)

air-side surface condition = wet
air out = 14.85 °C DB, 81 % RH
water out = 11.81 °C, 1.754 bar
heat transfer rate = 4.789 kW
heat exchanger effectiveness = 0.5612333030781438 fraction
air-side pressure drop = 12.961 Pa
water-side pressure drop = 24555.260 Pa


We can also get the heat transfer coefficients on the air and water side:

In [8]:
print(
    f"air-side heat transfer coefficient = "
    f"{air_coil.core.h_ext.to('W / (m ** 2 * K)'):~P.2f}",
    f"water-side heat transfer coefficient = "
    f"{air_coil.core.h_int.to('W / (m ** 2 * K)'):~P.2f}",
    sep='\n'
)

air-side heat transfer coefficient = 55.56 W/K/m²
water-side heat transfer coefficient = 10357.17 W/K/m²
